In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
import glob
# Read in and make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

nx = 9 # the number of inside corners in x
ny = 6 # the number of inside corners in y

# Arrays to store object points and image points
objpoints = [] #3d points in real world space
imgpoints = [] #2d points in image plane

# Prepare obj points, like (0,0,0), (1,0,0), (2,0,0), ... ,(8,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2) # x, y coordinates

for fname in images:
    #read in each image
    img = mpimg.imread(fname)
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If corners are found, add object points, image points
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (1280, 720), None, None)

In [3]:
def cal_undistort(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

def perspective_transform(calibrated):
    img_size = (calibrated.shape[1],  calibrated.shape[0])
    src = np.float32([[735, 460], [1110, 680], [290, 680], [600, 460]])
    dst = np.float32([[1110, 0], [1110, img_size[1]], [280, img_size[1]], [280, 0]])
    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)
    # Warp the image using OpenCV warpPerspective()
    warped = cv2.warpPerspective(calibrated, M, img_size)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return warped, Minv

def thresholded_binary(transformed, sx_thresh=(20, 100), s_thresh=(170, 255), b_thresh=(155, 200), l_thresh=(225, 255)):
    # Convert image to different schemes
    lab = cv2.cvtColor(transformed, cv2.COLOR_RGB2Lab)
    b_channel = lab[:,:,2]
    hls = cv2.cvtColor(transformed, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    luv = cv2.cvtColor(transformed, cv2.COLOR_RGB2LUV)
    l_channel = luv[:,:,0]

    gray = cv2.cvtColor(transformed, cv2.COLOR_BGR2GRAY)
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1    

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh[0]) & (b_channel <= b_thresh[1])] = 1

    combined = np.zeros_like(sxbinary)
    combined[(sxbinary == 1) | (b_binary == 1) | (l_binary == 1)] = 1

    return combined

def search_around_existing_poly(binary_warped):
    # Choose the width of the margin around the previous polynomial to search
    margin = 50

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Set the area of search based on activated x-values
    # within the +/- margin of polynomial function
    left_lane_inds = (nonzerox > (self.left_fit[0]*(nonzeroy**2) + self.left_fit[1]*nonzeroy + self.left_fit[2] - margin)) & (nonzerox < (self.left_fit[0]*(nonzeroy**2) + self.left_fit[1]*nonzeroy + self.left_fit[2] + margin))
    right_lane_inds = (nonzerox > (self.right_fit[0]*(nonzeroy**2) + self.right_fit[1]*nonzeroy + self.right_fit[2] - margin)) & (nonzerox < (self.right_fit[0]*(nonzeroy**2) + self.right_fit[1]*nonzeroy + self.right_fit[2] + margin))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 50
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                         (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # If > minpix pixels, recenter next window
        # (`right` or `leftx_current`) on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty

def fit_polynomial(leftx, lefty, rightx, righty, ploty):
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    global frame_num
    global last_known_good_left_fit
    global last_known_good_right_fit
    ## Check error between current coefficient and on from previous frame
    if frame_num == 0:
        last_known_good_left_fit = left_fit
        last_known_good_right_fit = right_fit
        frame_num = 1

    err_L = np.sum((left_fit[0] - last_known_good_left_fit[0])**2)
    err_L = np.sqrt(err_L)

    if err_L > .0005:
        left_fit = last_known_good_left_fit
    else:
        left_fit = .05*left_fit + .95*last_known_good_left_fit

    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        last_known_good_left_fit = left_fit
    except TypeError:
        left_fit = None
        left_fitx = last_known_good_left_fit[0]*ploty**2 + last_known_good_left_fit[1]*ploty +  last_known_good_left_fit[2]

    ## Check error between current coefficient and on from previous frame
    err_R = np.sum((right_fit[0] - last_known_good_left_fit[0]**2))
    err_R = np.sqrt(err_R)
    if err_R > .0005:
        right_fit = last_known_good_right_fit
    else:
        right_fit = .05*right_fit + .95*last_known_good_right_fit

    try:
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        last_known_good_right_fit = right_fit
    except TypeError:
        right_fit = None
        right_fitx = last_known_good_right_fit[0]*ploty**2 + last_known_good_right_fit[1]*ploty + last_known_good_right_fit[2]

    # offset calculation
    y_eval = np.max(ploty)

    bottom_left_x = left_fit[0]*y_eval**2 + left_fit[1]*y_eval + left_fit[2] if left_fit is not None else 0
    bottom_right_x = right_fit[0]*y_eval**2 + right_fit[1]*y_eval + right_fit[2] if right_fit is not None else 0
    center = (bottom_left_x + bottom_right_x) / 2
    offset = (1280/2 - center) * (3.7/700)

    return left_fitx, right_fitx, ploty, offset


def measure_curvature(fit_coeffs, y_eval):
    if fit_coeffs is not None and fit_coeffs[0] != 0:
        return ((1 + (2 * fit_coeffs[0] * y_eval + fit_coeffs[1]) ** 2) ** (3/2)) / abs(2 * fit_coeffs[0])
    else:
        raise Exception("invalid")

def measure_curvature_real(leftx, lefty, rightx, righty, ploty):
    '''
    Calculates the curvature of polynomial functions in meters.
    '''
    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 15.0/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    y_eval = np.max(ploty) * ym_per_pix
    left_fit_cr = np.polyfit(lefty * ym_per_pix, leftx * xm_per_pix, 2) 
    try:
        left_curverad = measure_curvature(left_fit_cr, y_eval)
        last_known_good_left_curverad = left_curverad
    except:
        left_curverad = last_known_good_left_curverad

    right_fit_cr = np.polyfit(righty * ym_per_pix, rightx * xm_per_pix, 2) 
    try:    
        right_curverad = measure_curvature(right_fit_cr, y_eval)
        last_known_good_right_curverad = right_curverad
    except:
        right_curverad = last_known_good_right_curverad

    return (left_curverad + right_curverad) / 2


def find_lane_lines(binary_warped):
    # sanity check
    if left_fit is not None and right_fit is not None:
        leftx, lefty, rightx, righty = search_around_existing_poly(binary_warped)
    else:
        leftx, lefty, rightx, righty = find_lane_pixels(binary_warped)

    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx, right_fitx, ploty, offset = fit_polynomial(leftx, lefty, rightx, righty, ploty)
    avg_curvature = measure_curvature_real(leftx, lefty, rightx, righty, ploty)

    return  left_fitx, right_fitx, avg_curvature, ploty, offset

def project_lane_detection_back_to_image(thresholded_binary_image, left_fitx, right_fitx, ploty, Minv, calibrated):
    shape = thresholded_binary_image.shape
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(thresholded_binary_image).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (shape[1], shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(calibrated, 1, newwarp, 0.3, 0)
    return result

In [4]:
def process_image_pipeline(image):
    # Polynomial fit values from the previous frame
    
    global left_fit
    global right_fit
    global last_known_good_left_curverad
    global last_known_good_right_curverad
    
    undistorted = cal_undistort(image, mtx, dist)
    transformed, Minv = perspective_transform(undistorted)
    thresholded_binary_image = thresholded_binary(transformed)
    left_fitx, right_fitx, avg_curvature, ploty, offset = find_lane_lines(thresholded_binary_image)
    result = project_lane_detection_back_to_image(thresholded_binary_image, left_fitx, right_fitx, ploty, Minv, undistorted)
    cv2.putText(result, f"radius of curvature = {avg_curvature} m", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    cv2.putText(result, f"offset = {offset} m", (30, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
    return result

In [5]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML


frame_num = 0
left_fit = None
right_fit = None
output = 'project_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
clip1 = VideoFileClip("project_video.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(39, )

white_clip = clip1.fl_image(process_image_pipeline)  #NOTE: this function expects color images!!
%time white_clip.write_videofile(output, audio=False)

[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████▉| 1260/1261 [04:55<00:00,  4.29it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

CPU times: user 2min 18s, sys: 26.8 s, total: 2min 45s
Wall time: 4min 58s


In [6]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))